# Play Doom Level using Trained PPO Agent  

In [1]:
from utils.env import make_vizdoom_env
from agents.doom_ppo_agent import DoomPpoAgent
import gymnasium as gym
import torch

# Creating Environment
num_envs = 1
envs = gym.vector.SyncVectorEnv([ make_vizdoom_env(level='vizdoom/scenarios/basic.cfg') for i in range(num_envs)])

# Prefering GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.has_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Setting up agent
agent = DoomPpoAgent(envs.single_observation_space, 
                     envs.single_action_space)
agent.to(device)
agent.load_models(path='./models/doom_ppo_agent/training_run_2023_07_07_02_24_27/checkpoint_step_292864', device=device)
agent.load_models(path='./models/doom_ppo_agent/training_run_2023_07_09_16_55_38/checkpoint_step_38912', device=device)

# Playing config
number_of_episodes = 50

/Users/harish/Documents/Study/University of London/Semester 6/Final Project/doom-rlhf/envs/vizdoom_env.py:82: UserWarning: Detected screen format CRCGCB. Only RGB24 and GRAY8 are supported in the Gymnasium wrapper. Forcing RGB24.
  warnings.warn(
/Users/harish/opt/anaconda3/envs/doom-rlhf/lib/python3.9/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(


Loading models...
Successfully loaded models!
Updating networks with weights from loaded models...


RuntimeError: Error(s) in loading state_dict for Linear:
	size mismatch for weight: copying a param with shape torch.Size([3, 512]) from checkpoint, the shape in current model is torch.Size([4, 512]).
	size mismatch for bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([4]).

In [ ]:
import time
import torch
import numpy as np

for episode in range(number_of_episodes):
    observation = envs.reset()
    done = np.zeros(num_envs, dtype=np.int32)
    total_reward = 0

    while not done.all():
        # Getting next action and it's value
        with torch.no_grad():
            action, log_prob, _, value = agent.forward(torch.Tensor(observation).to(device))
        observation, reward, done, info = envs.step(action.cpu().numpy())
        total_reward += reward

        time.sleep(1/30)

    print(f"Total reward for episode {episode} is {total_reward}")
    time.sleep(0.25)

In [ ]:
envs.close()